In [1]:
import os
import numpy as np

import open3d as o3d
from plyfile import PlyData, PlyElement

import matplotlib.pyplot as plt

import MinkowskiEngine as ME

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Load the point cloud
Use scan from ScanNet for the test

In [2]:
# scan_ply_filepath = '/home/rsl_admin/openscene/test_pcd/scene0000_00_vh_clean.ply'
scan_ply_filepath = '/home/rsl_admin/openscene/test_pcd/scene0000_00_vh_clean_2.ply'  # low res

ply_data = PlyData.read(scan_ply_filepath)

In [3]:
num_points = ply_data['vertex'].count

locs_np = np.zeros((num_points,3))
locs_np[:,0] = ply_data['vertex']['x']
locs_np[:,1] = ply_data['vertex']['y']
locs_np[:,2] = ply_data['vertex']['z']

colors_np = np.zeros((num_points,3))
colors_np[:,0] = ply_data['vertex']['red'] / 255.0
colors_np[:,1] = ply_data['vertex']['green'] / 255.0
colors_np[:,2] = ply_data['vertex']['blue'] / 255.0

In [4]:
print(locs_np.shape)
print(colors_np.shape)

(81369, 3)
(81369, 3)


In [5]:
# visualizate with open3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(locs_np)
pcd.colors = o3d.utility.Vector3dVector(colors_np)
o3d.visualization.draw_geometries([pcd])

## Create two point clouds, with some overlap

In [6]:
section_a_xlim = [-float('inf'), 5.0]
section_b_xlim = [2.0, float('inf')]

section_a_idx = np.logical_and(locs_np[:, 0] > section_a_xlim[0], locs_np[:, 0] < section_a_xlim[1])
section_b_idx = np.logical_and(locs_np[:, 0] > section_b_xlim[0], locs_np[:, 0] < section_b_xlim[1])

a_locs_np = locs_np[section_a_idx].copy()
a_colors_np = colors_np[section_a_idx].copy()

b_locs_np = locs_np[section_b_idx].copy()
b_colors_np = colors_np[section_b_idx].copy()

In [7]:
# section_a_ylim = [-float('inf'), 5.0]
# section_b_ylim = [2.0, float('inf')]

# section_a_idx = np.logical_and(locs_np[:, 1] > section_a_ylim[0], locs_np[:, 1] < section_a_ylim[1])
# section_b_idx = np.logical_and(locs_np[:, 1] > section_b_ylim[0], locs_np[:, 1] < section_b_ylim[1])

# a_locs_np = locs_np[section_a_idx].copy()
# a_colors_np = colors_np[section_a_idx].copy()

# b_locs_np = locs_np[section_b_idx].copy()
# b_colors_np = colors_np[section_b_idx].copy()

In [8]:
# visualizate the two point clouds
a_pcd = o3d.geometry.PointCloud()
a_pcd.points = o3d.utility.Vector3dVector(a_locs_np)
a_pcd.colors = o3d.utility.Vector3dVector(a_colors_np)
o3d.visualization.draw_geometries([a_pcd])

In [9]:
b_pcd = o3d.geometry.PointCloud()
b_pcd.points = o3d.utility.Vector3dVector(b_locs_np)
b_pcd.colors = o3d.utility.Vector3dVector(b_colors_np)
o3d.visualization.draw_geometries([b_pcd])

## Create voxelized representation for both

In [10]:
VOXEL_SIZE = 0.02
EMBED_SIZE = 768

By voxelized representation I mean coords and embeddings. For embeddings, we can just generated random numbers

In [11]:
from dataset.voxelization_utils import sparse_quantize

In [12]:
a_coords = np.floor(a_locs_np / VOXEL_SIZE)
a_unique_map, a_inverse_map = sparse_quantize(a_coords, return_index=True)
a_unique_coords = a_coords[a_unique_map]

# a_unique_embeds = np.random.randn(a_unique_coords.shape[0], EMBED_SIZE).astype(np.float32)

b_coords = np.floor(b_locs_np / VOXEL_SIZE)
b_unique_map, b_inverse_map = sparse_quantize(b_coords, return_index=True)
b_unique_coords = b_coords[b_unique_map]

# b_unique_embeds = np.random.randn(b_unique_coords.shape[0], EMBED_SIZE).astype(np.float32)

print(a_coords.shape)
print(a_unique_coords.shape)

print(b_coords.shape)
print(b_unique_coords.shape)

(49828, 3)
(48891, 3)
(64509, 3)
(63358, 3)


After under going sparse quantize both the unique coords should actually be unique, let's verify

In [13]:
assert np.unique(a_unique_coords, axis=0).shape[0] == a_unique_coords.shape[0]
assert np.unique(b_unique_coords, axis=0).shape[0] == b_unique_coords.shape[0]

### Let's find the overlaps between the two arrays

####
ASIDE: using `sparse_quantize` is much faster than calling `np.unqiue` even though `sparse_quantize` internally also calls `np.unique`, think this is due to `sparse_quantize` calling unique on a hashed array of type `uint64`, so it's only doing unique over hash ids instead of XYZ coords

In [14]:
# %timeit np.unique(cat_coords.astype(np.float32), axis=0, return_index=True)   # 285 ms
# %timeit np.unique(cat_coords.astype(np.int32), axis=0, return_index=True)     # 280 ms
# %timeit np.unique(cat_coords.astype(np.int16), axis=0, return_index=True)     # 282 ms

# %timeit cat_unique_map, cat_inverse_map = sparse_quantize(cat_coords, return_index=True) # 20ms

#### 
instead of relying on `sparse_quantize` let's just copy over the hash function

In [15]:
def fnv_hash_vec(arr):
    '''
    FNV64-1A
    '''
    assert arr.ndim == 2
    # Floor first for negative coordinates
    arr = arr.copy()
    arr = arr.astype(np.uint64, copy=False)
    hashed_arr = np.uint64(14695981039346656037) * \
                 np.ones(arr.shape[0], dtype=np.uint64)
    for j in range(arr.shape[1]):
        hashed_arr *= np.uint64(1099511628211)
        hashed_arr = np.bitwise_xor(hashed_arr, arr[:, j])
    return hashed_arr

In [16]:
def coords_overlap(a_unique_coords: np.ndarray, b_unique_coords: np.ndarray):
    '''
    Finds overlap between two sets of voxel coordinates
    
    The caller is responsible for ensuring that the arguments only contain
    unique coords
    
    Args:
        a_unique_coords: (N, 3) voxel coords, assumed to only contain unique coords
        b_unique_coords: (M, 3) voxel coords, assumed to only contain unique coords
        
    Returns:
        a_overlap: (N,) boolean array, elements are marked True if overlapping
        b_overlap: (M,) boolean array, elements are marked True if overlapping
    '''
    assert a_coords.ndim == 2 and b_coords.ndim == 2

    a_keys = fnv_hash_vec(a_unique_coords)
    b_keys = fnv_hash_vec(b_unique_coords)

    a_overlap = np.isin(a_keys, b_keys)
    b_overlap = np.isin(b_keys, a_keys)
    
    return a_overlap, b_overlap

In [17]:
# %timeit coords_overlap(a_unique_coords, b_unique_coords)

a_overlap_mask, b_overlap_mask = coords_overlap(a_unique_coords, b_unique_coords)

In [18]:
a_overlap_locs_np = a_unique_coords[a_overlap_mask] * VOXEL_SIZE
a_nonoverlap_locs = a_unique_coords[~a_overlap_mask] * VOXEL_SIZE

b_overlap_locs_np = b_unique_coords[b_overlap_mask] * VOXEL_SIZE
b_nonoverlap_locs_np = b_unique_coords[~b_overlap_mask] * VOXEL_SIZE

In [19]:
a_overlap_pcd = o3d.geometry.PointCloud()
a_overlap_pcd.points = o3d.utility.Vector3dVector(a_overlap_locs_np)
o3d.visualization.draw_geometries([a_overlap_pcd])

In [20]:
b_overlap_pcd = o3d.geometry.PointCloud()
b_overlap_pcd.points = o3d.utility.Vector3dVector(b_overlap_locs_np)
o3d.visualization.draw_geometries([b_overlap_pcd])

In [21]:
a_nonoverlap_pcd = o3d.geometry.PointCloud()
a_nonoverlap_pcd.points = o3d.utility.Vector3dVector(a_nonoverlap_locs)
o3d.visualization.draw_geometries([a_nonoverlap_pcd])

In [22]:
b_nonoverlap_pcd = o3d.geometry.PointCloud()
b_nonoverlap_pcd.points = o3d.utility.Vector3dVector(b_nonoverlap_locs_np)
o3d.visualization.draw_geometries([b_nonoverlap_pcd])

### Converting voxel size

In [32]:
def convert_voxel_size(coords, old_voxel_size, new_voxel_size, mode='take-one'):
    new_coords = np.floor(coords * old_voxel_size / new_voxel_size)
    new_keys = fnv_hash_vec(new_coords)

    unique_keys, inds = np.unique(new_keys, return_index=True)
    
    new_feat = np.zeros((len(unique_keys), 100))
    for key in unique_keys:
        inds = np.where(new_keys == key)[0]
        # print(inds)
    

In [33]:
%timeit convert_voxel_size(a_unique_coords, VOXEL_SIZE, 0.05)

355 ms ± 3.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
